In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import sys
sys.path.append('/app')

In [50]:
from vexa import VexaAuth

auth = VexaAuth()

In [51]:

from vexa import VexaAPI
vexa = VexaAPI()
user_id = (await vexa.get_user_info())['id']

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [58]:
from chat import SearchContextProvider
from search import SearchAssistant,ParsedSearchRequest
from chat import SearchChatManager

In [53]:

search_assistant = SearchAssistant()
context_provider = SearchContextProvider(search_assistant)

16:14:35 - vexa_api - INFO - Initializing SearchAssistant


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [55]:
r = await context_provider.get_context(user_id=user_id,query="маркетинг")

In [56]:
r

"| topic_name                                                                         | speaker_name      | summary                                                                                                                                                                   | details                                                                                                                                                                                                                                                                                                                |   relevance_score | datetime                   |   meeting_index |\n|:-----------------------------------------------------------------------------------|:------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------

In [60]:
chat_manager = SearchChatManager()

16:16:08 - vexa_api - INFO - Initializing ChatManager


In [65]:
from helpers import jupyter_stream_output
from typing import Optional

In [74]:
from typing import Optional
from helpers import jupyter_stream_output
from search import SearchAssistant

# Initialize managers
search_assistant = SearchAssistant()
chat_manager = SearchChatManager()

16:22:22 - vexa_api - INFO - Initializing SearchAssistant


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

16:22:24 - vexa_api - INFO - Initializing ChatManager


In [79]:

@jupyter_stream_output(markdown=True)
async def stream_chat(user_id: str, query: str, thread_id: Optional[str] = None):
    async for result in chat_manager.chat(
        user_id=user_id,
        query=query,
        search_assistant=search_assistant,
        thread_id=thread_id
    ):
        if 'chunk' in result:
            yield result['chunk']
        elif 'output' in result:
            yield result['output']

# Usage
await stream_chat(user_id=user_id, query="андрей")

Андрей упоминается как бэкэндер, с которым Дмитрий работал над проектами. Обсуждается его уход в банк и влияние на проект[94].Андрей упоминается как бэкэндер, с которым Дмитрий работал над проектами. Обсуждается его уход в банк и влияние на проект[94](54b5c296-4fa6-4708-bf90-e4524485b7a7).

'Андрей упоминается как бэкэндер, с которым Дмитрий работал над проектами. Обсуждается его уход в банк и влияние на проект[94].Андрей упоминается как бэкэндер, с которым Дмитрий работал над проектами. Обсуждается его уход в банк и влияние на проект[94](54b5c296-4fa6-4708-bf90-e4524485b7a7).'

In [71]:

# Call the decorated function
await stream_chat(user_id=user_id, query="маркетинг")

TypeError: SearchChatManager.chat() got an unexpected keyword argument 'context_kwargs'

In [67]:
await stream_chat(user_id=user_id,query="маркетинг")

TypeError: SearchContextProvider.get_context() missing 2 required positional arguments: 'user_id' and 'query'

In [59]:
SearchChatManager

chat.SearchChatManager

In [21]:
r = await ParsedSearchRequest.parse_request("маркетинг")
queries = r.search_queries

In [27]:
queries

[SearchQuery(query='маркетинг'),
 SearchQuery(query='маркетинговая стратегия'),
 SearchQuery(query='маркетинговые кампании'),
 SearchQuery(query='маркетинговые исследования'),
 SearchQuery(query='маркетинг в социальных сетях'),
 SearchQuery(query='цифровой маркетинг'),
 SearchQuery(query='брендинг и маркетинг'),
 SearchQuery(query='анализ рынка и маркетинг'),
 SearchQuery(query='тренды в маркетинге'),
 SearchQuery(query='маркетинг и продажи')]

In [40]:
search_results = [
            await search_assistant.search(q.query, user_id=user_id) 
            for q in queries
        ]

In [41]:
import pandas as pd

In [42]:
search_results = pd.concat(search_results)
search_results = search_results.drop(columns=['vector_scores', 'exact_matches'])\
    .drop_duplicates(subset=['topic_name', 'speaker_name', 'summary', 'details', 'meeting_id'])

# Prepare context using search assistant's methods
prepared_df, indexed_meetings = search_assistant.prep_context(search_results)

In [43]:
columns_to_drop = ['timestamp', 'vector_scores', 'exact_matches', 'source', 'score', 'meeting_id']
existing_columns = [col for col in columns_to_drop if col in prepared_df.columns]
context_prepared = prepared_df.drop(columns=existing_columns)

context = context_prepared.to_markdown(index=False) if not prepared_df.empty else "No relevant context found."
        

In [45]:
context = await context_provider._get_raw_context(user_id=user_id,query="маркетинг")

In [46]:
context

"| topic_name                                                                         | speaker_name      | summary                                                                                                                                                                   | details                                                                                                                                                                                                                                                                                                                |   relevance_score | datetime                   |   meeting_index |\n|:-----------------------------------------------------------------------------------|:------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------

In [47]:
context_provider.url_dict

dict_items([(1, '288381d0-3ca7-4601-99b9-b42da8cd30d3'), (2, 'a2926b2d-f4eb-4bcc-864f-27607faa39fb'), (3, '5c63474d-b8a9-4e76-af92-a8d646097ae9'), (4, 'c789fe81-bde1-4d8c-9aad-2c3f64bc9c28'), (5, '40001390-7187-47a7-9284-a1dab5dc0df6'), (6, '3630910d-a5e8-4f83-8b4e-bc0094c3987a'), (7, '4ff12d56-f520-4391-a283-03d6365b49e6'), (8, 'e6ce439d-5802-4914-8271-4fbcea74ee94'), (9, '7122979f-9b4e-4d0f-92c7-18f6b1034197'), (10, '2e36140e-e62e-4657-bbb4-a0473dbf608e'), (11, '081d04c6-827d-4375-af68-9f8fbd50b707'), (12, 'db7e69e1-35f4-4d8e-a018-1d6c705b1fe3'), (13, 'f7515f24-af43-4f07-9932-21255978d204'), (14, 'db0e1e1a-8c83-4d8d-8ac5-c7d4a8264242'), (15, 'bf174837-0662-4226-90bf-ca18ad302ae6'), (16, 'dbf8ad27-649c-4a16-b5a1-5ab3db002bd9'), (17, '6e6e65ac-b626-4326-aab4-f2a8e9b0cdef'), (18, 'f5f969d6-675f-4d55-a604-78208e545dcd'), (19, '1c71070f-4427-4aae-ace0-2091f70dec5b'), (20, '3540f190-1f83-4100-8d6f-e648752f336d'), (21, '04de7398-49fd-4a57-962f-2798fc080b04'), (22, '9178c8e7-da6a-4883-9fe1-c

In [14]:
await search_assistant.search(query="vexa",user_id=user_id)

,topic_name,speaker_name,summary,details,meeting_id,timestamp,score,vector_scores,exact_matches,source
1,Vexa,TS,"Vexa - это инструмент, который обсуждается как...",Татьяна интересуется возможностью использовани...,54b5c296-4fa6-4708-bf90-e4524485b7a7,2024-11-19T12:32:14,1.367346,"{'topic_name': 0.6836729500000001, 'speaker': ...","[topic_name, topic_name, topic_name]",main
2,Vexa,DG,A tool for generating content from meeting tra...,Dmitry suggests using Vexa to create newslette...,3540f190-1f83-4100-8d6f-e648752f336d,2024-09-26T12:02:02,1.367346,"{'topic_name': 0.6836729500000001, 'speaker': ...","[topic_name, topic_name]",main
3,Vexa,RH,A real-time assistant for meetings that helps ...,Robert discusses his positive experiences usin...,4ff12d56-f520-4391-a283-03d6365b49e6,2024-09-09T10:46:10,1.367346,"{'topic_name': 0.6836729500000001, 'speaker': ...","[topic_name, topic_name]",main
0,Vexa,DG,Vexa is mentioned as a product that Dmitry has...,Dmitry mentions trying Vexa for a meeting but ...,50d5d659-3a57-42f2-8f32-31c7145ebe91,2024-10-24T09:32:32,1.367346,"{'topic_name': 0.6836729500000001, 'speaker': ...","[topic_name, topic_name, topic_name]",main
4,Обсуждение возможности проведения курсов по Vexa,SR,Обсуждение идеи проведения курсов по Vexa для ...,"Сергей предложил, что если сделать Vexa showca...",553fcfe6-0143-4971-99e7-213245f63347,2024-10-30T14:13:33,1.249777,{'speaker': 0.6248883700000001},[],main
...,...,...,...,...,...,...,...,...,...,...
159,конверсии,DG,Обсуждение текущих показателей конверсии и акт...,Говорят о снижении конверсий и необходимости у...,40001390-7187-47a7-9284-a1dab5dc0df6,2024-09-05T18:45:16.340000,0.836876,{'topic_name': 0.8368758},[],main
161,рекомендации,IS,"Советы, предоставляемые в рамках консультаций.","Обсуждается, как TLDV может помочь в формирова...",bf174837-0662-4226-90bf-ca18ad302ae6,2024-09-13T11:30:15,0.835429,{'topic_name': 0.8354289},[],main
160,рекомендации,IS,"Советы, предоставляемые на основе анализа данных.",Обсуждается возможность создания рекомендаций ...,bf174837-0662-4226-90bf-ca18ad302ae6,2024-09-13T11:30:15,0.835429,{'topic_name': 0.8354289},[],main
163,Robert Hangu,RH,Consultant and speaker in the meeting sharing ...,Robert provides feedback on Vexa's interface a...,4ff12d56-f520-4391-a283-03d6365b49e6,2024-09-09T10:46:10,0.835272,"{'speaker': 0.54662255, 'topic_name': 0.47232505}",[],main


In [11]:
context = await context_provider.get_context(user_id=user_id,query="vexa")

/app/search.py:169: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)


In [12]:
context

''

In [ ]:

meetings = await vexa.get_meetings()

In [6]:
from chat import MeetingContextProvider,MeetingChatManager

In [7]:
from psql_helpers import get_session,get_meeting_by_id

In [8]:
from helpers import jupyter_stream_output

In [21]:
meeting_id = 'dbf8ad27-649c-4a16-b5a1-5ab3db002bd9'

In [20]:
from uuid import UUID
meeting_id = UUID(meeting_id)

AttributeError: 'UUID' object has no attribute 'replace'

In [22]:
meeting_id

'dbf8ad27-649c-4a16-b5a1-5ab3db002bd9'

In [23]:
query = 'summary'

In [24]:
thread_id = None

In [25]:
from prompts import Prompts
prompts = Prompts()
prompt = prompts.single_meeting_2711

In [29]:
context_provider = MeetingContextProvider([meeting_id])

In [34]:
async with get_session() as session:
    context = await context_provider.get_context(session=session)
context


Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


'Meeting Metadata:\nStart Date and Time: 2024-09-13 15:15:37\nSpeakers (Always  reference the names EXACTLY as provided with NO changes letter by letter):\n  DG: Dmitry Grankin\n  DC: Dmitrii Chistov\n\nMeeting Transcript:\n\nDC (0):  Все хорошо.\nDG (1):  Я хотел полюбопытствовать на тему, или пофантазировать на тему.\nDC (2):  это это твоя штука да сейчас запустилась векса то что\nDG (3):  пришло уведомляешь когда\nDC (4):  ну в этот чатик ушла и подосметили Ну да, это GDPR, хотя у нас есть кнопочка\nDG (5):  Выключить эту штуку, но, видимо, людей бесит не столько, не всегда транскрибируется звонок, сколько наличие.\nDC (6):  участник слышишь а это где кнопочка сама что-то не вижу то есть я вижу или или только ты можешь выключить я не могу но это же векса сама отправлять и могу отключить уведомления\nDG (7):  в чат\nDC (8):  Да, ты имеешь в виду, я понял.\nDG (9):  Да, ну вроде, нет, есть люди, у которых корпоративно просто запрещено, вот, и они натихаря записывают, тогда они отключа

In [35]:
@jupyter_stream_output(markdown=True)
async def chat_with_meeting(thread_id, user_id, meeting_id, query):
    async with get_session() as session:
        context_provider = MeetingContextProvider([meeting_id])
        meeting_chat_manager = MeetingChatManager(session, context_provider=context_provider)
        async for response in meeting_chat_manager.chat(
            meeting_ids=[meeting_id],
            user_id=user_id,
            query=query,
            thread_id=thread_id,
            temperature=0.7,
            prompt=prompts.single_meeting_2711
        ):
            yield response
# Usage
response = await chat_with_meeting(
    thread_id=thread_id,
    user_id='ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
    meeting_id=meeting_id,
    query="summare meeting"
)

- **Participants**: Dmitry Grankin (DG) and Dmitrii Chistov (DC)

- **Main Discussion Points**:
  
  - **Product and Marketing**:
    - They discussed the importance of separating marketing strategies from the product itself.
    - DC mentioned the need for a better understanding of user feedback to shape the product and its marketing approach.

  - **Traffic Generation**:
    - They explored methods for acquiring traffic, including using Google AdWords and the potential for social media advertising (e.g., Facebook Ads).
    - DC is considering testing different marketing campaigns to see which yields results.

  - **Outreach Strategies**:
    - DG emphasized the importance of building a strong LinkedIn profile and engaging with content to enhance outreach effectiveness.
    - They noted the need for personalized messaging to increase engagement and responses from potential clients.

  - **Customer Insights**:
    - DC shared his experience in gathering insights from active users to refine the product offerings.
    - They discussed how understanding user needs could inform product development.

  - **Market Competition**:
    - Both participants acknowledged the intense competition in the market, particularly with many companies offering similar solutions.
    - They discussed the challenges of differentiating their product in a crowded marketplace.

  - **Future Steps**:
    - They agreed to continue sharing insights and strategies to support each other in their marketing and sales efforts.
    - DC plans to further validate his product concept through outreach and potential user engagement.

- **Next Actions**:
  - DC will continue testing marketing campaigns and gather user feedback.
  - DG will send a presentation related to product-market fit that could help refine their approach.

In [12]:
meeting_

17:14:53 - vexa_api - INFO - Initializing ChatManager


In [13]:
await meeting_chat_manager.chat(user_id=user_id,query="What was the main topic of the meeting?",meeting_ids=[meeting_id])

<async_generator object MeetingChatManager.chat at 0x7f4aa5d6bba0>